In [1]:
import logging
import random

import numpy as np
import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# set seed 
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [2]:
# split data to 10 fold
fold_num = 10
data_file = './data/train_set.csv'
import pandas as pd


def all_data2fold(fold_num, num=10000):
    fold_data = []
    f = pd.read_csv(data_file, sep='\t', encoding='UTF-8')
    texts = f['text'].tolist()[:num]
    labels = f['label'].tolist()[:num]

    total = len(labels)

    index = list(range(total))
    np.random.shuffle(index)

    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])

    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)

    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels

        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


fold_data = all_data2fold(10)

2021-10-31 14:04:12,207 INFO: Fold lens [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [7]:
# build train data for word2vec
fold_id = 9

train_texts = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
    
logging.info('Total %d docs.' % len(train_texts))

2021-10-31 14:05:04,942 INFO: Total 9000 docs.


In [8]:
logging.info('Start training...')
from gensim.models.word2vec import Word2Vec

num_features = 100     # Word vector dimensionality
num_workers = 8       # Number of threads to run in parallel

train_texts = list(map(lambda x: list(x.split()), train_texts))
model = Word2Vec(train_texts, workers=num_workers, vector_size=num_features)
model.init_sims(replace=True)

# save model
model.save("./word2vec.bin")

2021-10-31 14:05:06,502 INFO: Start training...
2021-10-31 14:05:06,767 INFO: collecting all words and their counts
2021-10-31 14:05:06,771 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-31 14:05:07,394 INFO: collected 5295 word types from a corpus of 8191447 raw words and 9000 sentences
2021-10-31 14:05:07,394 INFO: Creating a fresh vocabulary
2021-10-31 14:05:07,424 INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 4335 unique words (81.86968838526913%% of original 5295, drops 960)', 'datetime': '2021-10-31T14:05:07.402322', 'gensim': '4.1.2', 'python': '3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:21:17) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'prepare_vocab'}
2021-10-31 14:05:07,425 INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 8189498 word corpus (99.97620688994265%% of original 8191447, drops 1949)', 'datetime': '2021-10-31T14:05:07.425832', 'gensim': '4.1.2'

In [9]:
# load model
model = Word2Vec.load("./word2vec.bin")

# convert format
model.wv.save_word2vec_format('./word2vec.txt', binary=False)

2021-10-31 14:05:29,173 INFO: loading Word2Vec object from ./word2vec.bin
2021-10-31 14:05:29,177 INFO: loading wv recursively from ./word2vec.bin.wv.* with mmap=None
2021-10-31 14:05:29,177 INFO: setting ignored attribute cum_table to None
2021-10-31 14:05:29,196 INFO: Word2Vec lifecycle event {'fname': './word2vec.bin', 'datetime': '2021-10-31T14:05:29.196779', 'gensim': '4.1.2', 'python': '3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:21:17) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'loaded'}
2021-10-31 14:05:29,199 INFO: storing 4335x100 projection weights into ./word2vec.txt
